<a href="https://colab.research.google.com/github/gabriellbragaa/Computer_Vision_Projs/blob/main/lab5resolu%C3%A7%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Usando com obase o código fornecido, apique o processo de homografia uando 3 imagens para montar um panorama.**

O processo de **Homografia** é a transformação geométrica que mapeia um ponto de um plano para outro plano. Ou seja, é uma maneira de "distorcer" uma imagem tirada em pespectiva para que pareça que foi tirada de frente, ou de alinhar duas imagens da mesma cena plana tirada em algulos diferente.

No código abaixo, a **homografia** é usada para alinhar as imagens. Primeiro, o código encontra pontos distintos e único em cada imagem "**Keypoints**" e descreve as caracteristicas, e em seguida, ele encontra quais os kaypoints correspondentes de uma imagem para outra. Logo após tudo isso, o código calcula a matriz de homografia.

- Utilizei no código o **sift = cv2.SIFT_create()** que é responsável por detectar e descrever os pontos distintos ( keypoints) em uma imagem.
- Utilizei o **BFmatcher** que compara o descritor de um ponto de interesse em uma imagem com todos os descritores de pontos de interesse na outra imagem e retorna a correspondência que tem a menor distância (ou seja, a mais semelhante).

In [ ]:

import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files


print("primeira imagem:")
uploaded1 = files.upload()
filename1 = list(uploaded1.keys())[0]
img1 = cv2.imread(filename1)

if img1 is None:
    print(f"Erro ao carregar a imagem: {filename1}")
else:
    plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
    plt.title('Primeira Imagem Carregada')
    plt.axis('off')
    plt.show()

print("Segunda imagem:")
uploaded2 = files.upload()
filename2 = list(uploaded2.keys())[0]
img2 = cv2.imread(filename2)

if img2 is None:
    print(f"Erro ao carregar a imagem: {filename2}")
else:
    plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
    plt.title('Segunda Imagem Carregada')
    plt.axis('off')
    plt.show()

print("Terceira imagem:")
uploaded3 = files.upload()
filename3 = list(uploaded3.keys())[0]
img3 = cv2.imread(filename3)

if img3 is None:
    print(f"Erro ao carregar a imagem: {filename3}")
else:
    plt.imshow(cv2.cvtColor(img3, cv2.COLOR_BGR2RGB))
    plt.title('Terceira Imagem Carregada')
    plt.axis('off')
    plt.show()

# Detecção e correspondencia de caractéristicas

# Converte para Cinza
img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
img3_gray = cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY)


sift = cv2.SIFT_create()

# Procura os keypoints e a descrição de cada imagem
kp1, des1 = sift.detectAndCompute(img1_gray, None)
kp2, des2 = sift.detectAndCompute(img2_gray, None)
kp3, des3 = sift.detectAndCompute(img3_gray, None)

# BFMatcher
bf = cv2.BFMatcher()
matches12 = bf.knnMatch(des1, des2, k=2)
matches23 = bf.knnMatch(des2, des3, k=2)

#  test para img1 e img2
good_matches12 = []
for m, n in matches12:
    if m.distance < 0.75 * n.distance:
        good_matches12.append(m)


#  test para img2 e img3
good_matches23 = []
for m, n in matches23:
    if m.distance < 0.75 * n.distance:
        good_matches23.append(m)

print(f" {len(good_matches12)}  matches entre a imagem 1 e a imagem 2.")
print(f" {len(good_matches23)}  matches entre a imagem 2 e a imagem 3.")


# Step 3: Calcular a Homography

# Extração dos bons matches da img1 e img2
if len(good_matches12) >= 4:
    pts1 = np.float32([kp1[m.queryIdx].pt for m in good_matches12]).reshape(-1, 1, 2)
    pts2 = np.float32([kp2[m.trainIdx].pt for m in good_matches12]).reshape(-1, 1, 2)

    # Calcula a homography matrix entre img1 e img2 usando RANSAC
    H12, mask12 = cv2.findHomography(pts1, pts2, cv2.RANSAC, 5.0)
    print("\nHomografia calculada entre Imagem 1 e Imagem 2:")
    print(H12)
else:
    print("\nNão há keypoints suficientes entre Imagem 1 e Imagem 2.")
    H12 = None

# 2 e 3
if len(good_matches23) >= 4:
    pts2_ = np.float32([kp2[m.queryIdx].pt for m in good_matches23]).reshape(-1, 1, 2)
    pts3 = np.float32([kp3[m.trainIdx].pt for m in good_matches23]).reshape(-1, 1, 2)


    H23, mask23 = cv2.findHomography(pts2_, pts3, cv2.RANSAC, 5.0)
    print("\nHomografia calculada entre Imagem 2 e Imagem 3:")
    print(H23)
else:
    print("\nNão há keypoints suficientes entre Imagem 2 e Imagem 3.")
    H23 = None


# Checar as metricas
if H12 is not None and H23 is not None:

    height1, width1 = img1.shape[:2]
    height2, width2 = img2.shape[:2]
    height3, width3 = img3.shape[:2]

    corners_img1 = np.float32([[0, 0], [0, height1], [width1, height1], [width1, 0]]).reshape(-1, 1, 2)
    transformed_corners1 = cv2.perspectiveTransform(corners_img1, H12)

    H32 = np.linalg.inv(H23)
    corners_img3 = np.float32([[0, 0], [0, height3], [width3, height3], [width3, 0]]).reshape(-1, 1, 2)
    transformed_corners3 = cv2.perspectiveTransform(corners_img3, H32)


    all_corners = np.concatenate((transformed_corners1, np.float32([[0, 0], [0, height2], [width2, height2], [width2, 0]]).reshape(-1, 1, 2), transformed_corners3), axis=0)

    min_x = int(np.min(all_corners[:, :, 0]))
    min_y = int(np.min(all_corners[:, :, 1]))
    max_x = int(np.max(all_corners[:, :, 0]))
    max_y = int(np.max(all_corners[:, :, 1]))


    translation_matrix = np.array([[1, 0, -min_x], [0, 1, -min_y], [0, 0, 1]], dtype=np.float32)


    H12_translated = translation_matrix @ H12
    H32_translated = translation_matrix @ H32
    H2_translated = translation_matrix


    panorama_width = max_x - min_x
    panorama_height = max_y - min_y


    panorama = cv2.warpPerspective(img1, H12_translated, (panorama_width, panorama_height))

    panorama = cv2.warpPerspective(img2, H2_translated, (panorama_width, panorama_height), dst=panorama, borderMode=cv2.BORDER_TRANSPARENT)

    panorama = cv2.warpPerspective(img3, H32_translated, (panorama_width, panorama_height), dst=panorama, borderMode=cv2.BORDER_TRANSPARENT)



    if panorama is not None:
        plt.figure(figsize=(20,10))
        plt.imshow(cv2.cvtColor(panorama, cv2.COLOR_BGR2RGB))
        plt.title('Panorama Resultante')
        plt.axis('off')
        plt.show()
    else:
        print("\nNão foi possível gerar o panorama.")

else:
    print("\nHomografia não calculada. Não é possível costurar as imagens.")
    panorama = None

primeira imagem:


KeyboardInterrupt: 